## Missing Persons DB Webscrape

https://www.services.rcmp-grc.gc.ca/missing-disparus/search-recherche.jsf

Search with no criteria will bring up all results.

The links to the missing persons pages, can be appended to https://www.services.rcmp-grc.gc.ca/

### ========== Code

#### Get all the links to the more detailed case pages from the navigation pages

In [31]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
link = 'https://www.services.rcmp-grc.gc.ca/missing-disparus/search-recherche.jsf'
from webdriver_manager.chrome import ChromeDriverManager
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(link)
time.sleep(2)

# find search and click it to reach data
try:
    search = browser.find_element_by_name('searchForm:j_idt158')
    time.sleep(2)
except:
    print('could not find Search')
    
try:
    search.click()
except:
    print('could not find Search')

time.sleep(5)

not_last = True
# to store all the URLs
URLs = []

# go through each page and get all URLs    
while not_last:
    time.sleep(2) # wait more just in case
    page = browser.page_source
    pageSoup = bs(page, 'html.parser')
    
    # get all the links on the page and add them to array
    pageURLs = pageSoup.find_all('a',class_="wet-newwindow")
    
    print("Collecting URLs...")
    
    # take each link on the page and add if not a dupe
    for link in pageURLs:
        href = link.get('href')
        if href not in URLs:
            URLs.append(href)

    print("Page Complete!")
    time.sleep(2) # wait a little
    
    # are we on the last page
    try:
        # click the next button at the bottom of the page
        next_page = browser.find_element_by_xpath('/html/body/main/form/div[33]/ul/li[83]/a')
        print('Found next button to press.')
        next_page.click()
        time.sleep(4) # wait for next page to load
    except:
        # should not have a next button on the last page
        print('last page or no next button found!')
        not_last = False

# the final list
print("================================== END ==================================")
print(len(URLs[:30])) # show a section       
time.sleep(5)
browser.quit()

#### Collect all the data from all the detailed case pages

In [ ]:
# loop through all the URLs and collect the data from each page
for url in URLs:
    persons_page_url = 'https://www.services.rcmp-grc.gc.ca' + url